<a href="https://colab.research.google.com/github/antonsysoev/DeepLearning/blob/main/Text_seq_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from keras.preprocessing.text import Tokenizer

In [45]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

In [46]:
tokenizer = Tokenizer(num_words=1000)

In [47]:
tokenizer.fit_on_texts(samples)

In [48]:
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

In [49]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


Прямое кодирование на уровне слов с использованием хеширования

In [51]:
dimensionality = 1000
max_length = 10

In [52]:
import numpy as np

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = abs(hash(word)) % dimensionality
    results[i, j, index] = 1.

In [54]:
results[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Загрузка данных из IMDB для передачи в слой Embedding

In [55]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.6 MB/s eta 0:00:00


In [60]:
from keras.datasets import imdb
import keras
max_features = 25
maxlen = 100

In [59]:
(x_train, y_train), (x_test, y_test) = imdb.load_data( num_words=max_features)

17464789/17464789 [==============================] - 0s 0us/step


In [61]:
x_train = keras.utils.data_utils.pad_sequences (x_train, maxlen=maxlen)
x_test = keras.utils.data_utils.pad_sequences (x_test, maxlen=maxlen)

In [64]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 8)            80000     
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 801       
                                                                 
Total params: 80,801
Trainable params: 80,801
Non-trainable params: 0
_________________________________________________________________


In [65]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 4s 4ms/step - loss: 0.6902 - acc: 0.5319 - val_loss: 0.6806 - val_acc: 0.5850
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.6684 - acc: 0.5957 - val_loss: 0.6597 - val_acc: 0.6072
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.6555 - acc: 0.6094 - val_loss: 0.6569 - val_acc: 0.6060
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6517 - acc: 0.6140 - val_loss: 0.6576 - val_acc: 0.6080
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.6492 - acc: 0.6188 - val_loss: 0.6574 - val_acc: 0.6086
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6473 - acc: 0.6188 - val_loss: 0.6580 - val_acc: 0.6042
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6455 - acc: 0.6245 - val_loss: 0.6614 - val_acc: 0.6062
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.

создадим модель со слоем LSTM 



In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0da56cf74b69586f1007dc207bb4e6ba810f4a0a98e8e5aab66f83185954d52c
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd  # для таблиц
import numpy as np   # для матриц

# визуализация 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

import re # библиотека для регулярных выражений

# лемматизатор
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# список стоп-слов
import nltk.corpus
nltk.download("stopwords")
stopwords = set([word for word in nltk.corpus.stopwords.words('russian') if word != 'не'])

# удобный словарь
from collections import defaultdict

# слои для нейронок, которые нам пригодятся 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [66]:
df_neg = pd.read_csv('negative.csv', sep=';',header=None)
df_pos = pd.read_csv('positive.csv', sep=';',header=None)

df = df_pos[[3,4]].append(df_neg[[3,4]])
df.columns = ['text', 'target']
df.reset_index(drop=True, inplace=True)
df.target = df.target.replace({-1:0})

print(df.shape)
df.head()

(226834, 2)


,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


В первом столбце тексты твиттов. Во втором столбце две метки: либо 
1
, если твит позитивный, либо 
0
, если твит негативный.

In [68]:
df.text.iloc[-10]  # пример позитива


'@ivanenko14 и у меня также, только будильник еще и не выключался.. папу разбудила ('

Займёмся предобработкой текстов.

очистим их от мусорных символов

лемматиизируем

выбросим стоп-слова

In [69]:
text = df.text[0] 
words = [morph.normal_forms(word)[0] for word in 
             re.sub('[^a-zа-я0-9]', ' ', text.lower()).split(' ') if (len(word) >= 2) and (word not in stopwords)]

' '.join(words)

'first timee школотый поверь самый общество профилировать предмет тип'

In [70]:
words

['first',
 'timee',
 'школотый',
 'поверь',
 'самый',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [71]:
def prepare_text(text):  
    words = [morph.normal_forms(word)[0] for word in 
             re.sub('[^a-zа-я0-9]', ' ', text.lower()).split(' ') if (len(word) >= 2) and (word not in stopwords)]
    return words

In [11]:
%%time
df['prepare_text'] = df.text.apply(prepare_text)

CPU times: user 11min 10s, sys: 2.46 s, total: 11min 12s
Wall time: 11min 28s


In [12]:
df.head()

,text,target,prepare_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,"[first, timee, школотый, поверь, самый, общест..."
1,"Да, все-таки он немного похож на него. Но мой ...",1,"[таки, немного, похожий, мальчик, равно]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,"[rt, katiacheh, идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"[rt, digger2912, угол, сидеть, погибать, голод..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"[irina, dyshkant, значит, страшилка, блин, пос..."


In [72]:
words_count = defaultdict(lambda: 0)

for text in df.prepare_text:
    for word in text:
        words_count[word] +=1
        
len(words_count)

AttributeError: ignored

In [14]:
top_words = 24999    # ограниичимся словарём из 25000 самых частых слов

sorted_words = sorted(words_count.items(), key=lambda w:w[1], reverse=True)
freq_words = {item[0] for item in sorted_words[:top_words]}

encoder = dict(zip(freq_words, list(range(1,len(freq_words) + 1))))

# функция-фильтр
def freq_filter(text):
    return np.array([encoder[word] for word in text if word in freq_words])

df['freq_text'] = df.prepare_text.apply(freq_filter)
df.head()

,text,target,prepare_text,freq_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,"[first, timee, школотый, поверь, самый, общест...","[19591, 4379, 11927, 7334, 2375, 15095, 18757]"
1,"Да, все-таки он немного похож на него. Но мой ...",1,"[таки, немного, похожий, мальчик, равно]","[8316, 17424, 1962, 3115, 24851]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,"[rt, katiacheh, идиотка, испугаться]","[11801, 2082, 9100]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"[rt, digger2912, угол, сидеть, погибать, голод...","[11801, 21342, 5420, 14495, 18296, 7120, 14741..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"[irina, dyshkant, значит, страшилка, блин, пос...","[10478, 23236, 3314, 8337, 12501, 20320, 19230..."


In [15]:
dff = df[df.freq_text.apply(len) > 2] # Выбрасываем из обучения все посты короче 3-х слов.

In [19]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=e25f1dfd7491a2a1e0d70c0ef2f8f733ace0c3fc5ef78499239132bf02a97e3b
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.freq_text.to_numpy(), df.target.to_numpy(), test_size=0.2)

In [25]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

# как выглядит вектор
X_train[0][:20]

181467 train sequences
45367 test sequences


array([11801,  7265,  8706, 20118,  2112, 15461, 10419,  2196])

In [32]:
max_twitter_length = 20         # обрежем все посты до 20 слов

X_train = keras.utils.data_utils.pad_sequences(X_train, maxlen=max_twitter_length)
X_test = keras.utils.data_utils.pad_sequences(X_test, maxlen=max_twitter_length)

Попробуем обучить несколько разных малокалиберных нейросеток.



In [33]:
embedding_vector_length = 64    # размерность эмбендинга
top = top_words + 1             # на вход идут слова и нулевой элемент

In [34]:
model = Sequential()
model.add(Embedding(top, embedding_vector_length, input_length=max_twitter_length))
model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# собираем сетку и выбираем оптимизатор для обучения
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
hist = model.fit(X_train, y_train, validation_split=0.15, epochs=1, batch_size=32)

4821/4821 [==============================] - 210s 42ms/step - loss: 0.5577 - accuracy: 0.7109 - val_loss: 0.5206 - val_accuracy: 0.7398


In [36]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.4f'%(scores[1]))

Accuracy: 0.7399


In [38]:
model = Sequential()
model.add(Embedding(top, embedding_vector_length, input_length=max_twitter_length))
model.add(LSTM(1, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(1))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
%%time
hist = model.fit(X_train, y_train, validation_split=0.2, epochs=1, batch_size=32)

4537/4537 [==============================] - 219s 47ms/step - loss: 0.6511 - accuracy: 0.5867 - val_loss: 0.5847 - val_accuracy: 0.7205
CPU times: user 4min 58s, sys: 13 s, total: 5min 11s
Wall time: 3min 38s


In [40]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.4f'%(scores[1]))

Accuracy: 0.7181


В принципе эксперименты можно ставить довольно долго. К сущесвенному улучшению качества это не приведёт. Для этого нужно растить выборку. Остановимся на последней архитектуре и перейдём к выбору порогов. Подберём два порога: для детекции позитива и детекции негатива. Всё, что между ними будем относить к нейтральному классу.

In [42]:
from sklearn.metrics import classification_report

cutoff = 0.65  # выбрали порог

# говорим, что все клиенты с вероятностью оттока больше 0.5 уйдут
y_hat = model.predict(X_test) > cutoff

# выводим на экран отчёт, в которым посчитаются метрики по всем классам
report = classification_report(y_test, y_hat)
print(report)

1418/1418 [==============================] - 11s 5ms/step
              precision    recall  f1-score   support

           0       0.57      0.94      0.71     22105
           1       0.85      0.33      0.48     23262

    accuracy                           0.63     45367
   macro avg       0.71      0.63      0.59     45367
weighted avg       0.72      0.63      0.59     45367



In [73]:
cutoff = 0.5  # выбрали погог

# говорим, что все клиенты с вероятностью оттока больше 0.5 уйдут
y_hat = model.predict(X_test) > cutoff

# выводим на экран отчёт, в которым посчитаются метрики по всем классам
report = classification_report(y_test, y_hat)
print(report)

ValueError: ignored